# OpenSearch Warming Up
## Ref: 
- OpenSearch Python Client
    - https://opensearch.org/docs/1.3/clients/python-high-level/

In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import print_ww



## OpenSearch Client 생성

In [12]:
from typing import List, Tuple
from opensearchpy import OpenSearch, RequestsHttpConnection

def create_aws_opensearch_client(region: str, host: str, http_auth: Tuple[str, str]) -> OpenSearch:
    '''
    오픈서치 클라이언트를 제공함.
    '''
    aws_client = OpenSearch(
        hosts = [{'host': host.replace("https://", ""), 'port': 443}],
        http_auth = http_auth,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection
    )
    
    return aws_client



aws_region = 'us-east-1'
opensearch_cluster_domain = 'https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com'

os.environ["OpenSearch_UserName"] = "<Type UserName>" 
os.environ["OpenSearch_UserPassword"] = "<Type Password>" 

rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]

http_auth = (rag_user_name, rag_user_password)

aws_client = create_aws_opensearch_client(
                                     aws_region,
                                     opensearch_cluster_domain,
                                     http_auth)
aws_client    

<OpenSearch([{'host': 'search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com', 'port': 443}])>

## Index Creation

In [13]:
def create_index(aws_client, index_name, index_body):    
    response = aws_client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)
    
    
# Create an index with non-default settings.
index_name = 'default-index'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 2
    }
  }
}
    
create_index(aws_client, index_name, index_body)    


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'default-index'}


## Check if index exists

In [10]:
def check_if_index_exists(aws_client, index_name):
    '''
    인덱스가 존재하는지 확인
    '''
    exists = aws_client.indices.exists(index_name)
    print(f"index_name={index_name}, exists={exists}")
    return exists


index_exists = check_if_index_exists(aws_client, index_name)
index_exists

index_name=default-index, exists=True


True

In [36]:
import json

def show_index_definition(aws_client, index_name):
    # Retrieve index information
    index_info = aws_client.indices.get(index=index_name)

    # Print the retrieved information
    print("Index Information for '{}':".format(index_name))

    
    return index_info
    
index_def = show_index_definition(aws_client, index_name)    
print(index_def)

json.dumps
# pretty_json = json.load(index_def)
# pretty_json = json.dumps(pretty_json, indent=4, seperator='\n' )
# pretty_json


Index Information for 'default-index':
{'default-index': {'aliases': {}, 'mappings': {}, 'settings': {'index': {'creation_date': '1695629898503', 'number_of_shards': '2', 'number_of_replicas': '1', 'uuid': 'OSNqbtm6TL2V7QNhY18k8g', 'version': {'created': '136287827'}, 'provided_name': 'default-index'}}}}


ValueError: not enough values to unpack (expected 2, got 1)

## Delete Index 

In [11]:
# Delete the index.

def delete_index(aws_client, index_name):
    response = aws_client.indices.delete(
        index = index_name
    )

    print('\nDeleting index:')
    print(response)
    
delete_index(aws_client, index_name)    


Deleting index:
{'acknowledged': True}
